# 朴素贝叶斯

[慕课课程](https://www.icourse163.org/learn/WHUT-1205885803?tid=1468973450#/learn/content?type=detail&id=1251679198&cid=1281257391)

![](https://picgo-1257938625.cos.ap-nanjing.myqcloud.com//picgo/%E5%A4%9A%E5%88%86%E7%B1%BB%E7%9A%84%E6%9C%B4%E7%B4%A0%E8%B4%9D%E5%8F%B6%E6%96%AF.png)

![](https://picgo-1257938625.cos.ap-nanjing.myqcloud.com//picgo/%E6%A8%A1%E5%BC%8F%E8%AF%86%E5%88%AB-4-%E5%A4%9A%E7%BB%B4%E6%AD%A3%E6%80%81%E5%88%86%E5%B8%83%E7%9A%84%E7%89%B9%E5%BE%81%E5%90%91%E9%87%8F.png)

## 基于最小错误率的Bayes分类器

### numpy实现naive bayes

● 已知三个类别的样本分别为：

● w1:[0,0],[2,1],[1,0]

● w2:[-1,1],[-2,0],[-2,-1]

● ω3：[0，-2]，[0，-1]，[1，-2]

● 画出三类的分界线，以及判断样本[-2,2]属于哪一类。

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

#构建数据集

x1=np.array([[0,0],[2,1],[1,0]])
x2 = np.array([[-1,1],[-2,0],[-2,-1]])
x3 = np.array([[0,-2],[0,-1],[1,-2]])

y1 = np.array([1]*len(x1))
y2 = np.array([2]*len(x2))
y3 = np.array([3]*len(x3))

# 将数据放入统一的训练集中

X = np.concatenate((x1,x2,x3),axis=0)
Y = np.concatenate((y1,y2,y3), axis=0)
print('训练样本的特征：{} \n 训练样本的标签：{}'.format(X,Y))


训练样本的特征：[[ 0  0]
 [ 2  1]
 [ 1  0]
 [-1  1]
 [-2  0]
 [-2 -1]
 [ 0 -2]
 [ 0 -1]
 [ 1 -2]] 
 训练样本的标签：[1 1 1 2 2 2 3 3 3]


$$P(\omega_i|x)=\frac{P(x|\omega_i)P(\omega_i)}{P(x)}$$

$$p(x|\mu, \sigma)=\frac{1}{\sigma \sqrt{2\pi}}e^{-\frac{(x-\mu)^2}{2\sigma ^2}}$$

In [18]:
# 训练

# 参数设定
eps = 1e-6
labels = np.unique(Y)   # [1 2 3]   所有标签的类别
K = len(labels) # 3 标签的数量
N, M = X.shape  # N=9, 样本维度M=2

# 高斯分布的参数初始化
parameters = {
    'mean': np.zeros((K, M)),   # 均值  dim=[2,3]
    'sigma': np.zeros((K,M)),   # 方差
    'prior': np.zeros((K,)),    # 先验概率
}

# 计算每个类别的均值 方差 先验概率

for i, c in enumerate(labels):  
    # c:1, 2, 3; Y:[1,1,1,2,2,2,3,3,3],通过 Y==c 取得数据集中的X_1, X_2, X_3  

    X_c = X[Y == c, :]  
    # i: 0, 1, 2
    parameters['mean'][i, :] = np.mean(X_c, axis=0)
    # +eps 防止方差为0
    parameters['sigma'][i, :] = np.var(X_c, axis=0) + eps 
    parameters['prior'][i] = X_c.shape[0] / N

In [19]:
# 预测

def predict(x, labels):
    K = len(labels)  # 3
    log_posterior = np.zeros((x.shape[0], K)) # 缓存后验概率
    #  每个类别的后验概率
    for i in range(K):
        mu = parameters['mean'][i]
        prior = parameters['prior'][i]
        sigsq = parameters['sigma'][i]

        # 代入对数化的判别公式计算

        # log likelyhood = log X / N(mu, sigsq)

        log_likelihood = -0.5 * np.sum(np.log(2 * np.pi * sigsq)) 
        log_likelihood -= 0.5 * np.sum(((x - mu) **2) / sigsq, axis=1)
        # g_i(x)
        log_posterior[:, i] = log_likelihood + np.log(prior)
    # 找出后验概率中最大概率对应的索引(类别)
    pred = labels[log_posterior.argmax(axis=1)]
    return pred, log_posterior

p = np.array([[-1, 0], ])
pred, log_posterior = predict(p, labels)
print(f'样本属于第{pred}类,','对数概率为：',log_posterior)

样本属于第[2]类, 对数概率为： [[ -5.23171548  -2.9817166  -11.68237033]]


### sklearn 实现朴素naive bayes

In [20]:
import numpy as np
import matplotlib.pyplot as plt

# import datasets, classifier and performance metrics
from sklearn import datasets, metrics
from sklearn.naive_bayes import GaussianNB

![](https://picgo-1257938625.cos.ap-nanjing.myqcloud.com//picgo/20221117111626.png)

In [22]:
# The digits dataset

digits = datasets.load_digits()
images_and_labels = list(zip(digits.images, digits.target))

# flatten the image
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
data = digits.images.reshape((n_samples, -1))

print('shape of data:{0}'.format(data.shape))


shape of data:(1797, 64)
